### 预备知识

#### 数据操作

##### 2.1 创建Tensor

In [1]:
import torch

In [2]:
x = torch.empty(5,3)
x

tensor([[4.6894e+27, 7.9463e+08, 3.2604e-12],
        [1.7743e+28, 2.0535e-19, 5.9682e-02],
        [7.0374e+22, 3.8946e+21, 4.4650e+30],
        [7.0975e+22, 7.9309e+34, 7.9439e+08],
        [3.2604e-12, 7.3113e+34, 2.0706e-19]])

In [3]:
x = torch.rand(5,3)
x

tensor([[0.1687, 0.9754, 0.8769],
        [0.1098, 0.0990, 0.1993],
        [0.0335, 0.2908, 0.5057],
        [0.5457, 0.7826, 0.7080],
        [0.4042, 0.4293, 0.5127]])

In [4]:
x = torch.zeros(5,3,dtype=torch.long)
x

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

In [5]:
x = torch.tensor([5.5,3],dtype=torch.float)
x

tensor([5.5000, 3.0000])

In [6]:
x = x.new_ones(5, 3, dtype=torch.float64)  # 返回的tensor默认具有相同的torch.dtype和torch.device
print(x)

x = torch.randn_like(x, dtype=torch.float) # 指定新的数据类型
print(x) 

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[-1.5320, -0.0186,  2.9297],
        [ 0.9589, -0.1601, -0.3159],
        [ 0.0076, -0.1867, -2.1629],
        [-0.5118, -1.2420,  0.5091],
        [ 0.0688,  0.8951, -1.1266]])


In [7]:
print(x.size())
print(x.shape)

torch.Size([5, 3])
torch.Size([5, 3])


##### 2.2 操作

加法：

In [8]:
y = torch.rand(5,3)
x + y

tensor([[-1.2957,  0.9340,  3.2519],
        [ 1.8556, -0.0193,  0.6420],
        [ 0.2661,  0.7964, -1.4619],
        [ 0.0874, -0.5633,  1.3453],
        [ 0.4716,  1.5962, -0.2711]])

In [9]:
torch.add(x,y)

tensor([[-1.2957,  0.9340,  3.2519],
        [ 1.8556, -0.0193,  0.6420],
        [ 0.2661,  0.7964, -1.4619],
        [ 0.0874, -0.5633,  1.3453],
        [ 0.4716,  1.5962, -0.2711]])

In [10]:
result = torch.empty(5, 3)
torch.add(x, y, out=result)
print(result)

tensor([[-1.2957,  0.9340,  3.2519],
        [ 1.8556, -0.0193,  0.6420],
        [ 0.2661,  0.7964, -1.4619],
        [ 0.0874, -0.5633,  1.3453],
        [ 0.4716,  1.5962, -0.2711]])


In [11]:
y.add_(x)
y

tensor([[-1.2957,  0.9340,  3.2519],
        [ 1.8556, -0.0193,  0.6420],
        [ 0.2661,  0.7964, -1.4619],
        [ 0.0874, -0.5633,  1.3453],
        [ 0.4716,  1.5962, -0.2711]])

#### 索引    
注意：索引出来的结果与原数据共享内存，也即修改一个，另一个会跟着修改。

In [12]:
y = x[0, :]
y += 1
print(y)
print(x[0, :]) # 源tensor也被改了

tensor([-0.5320,  0.9814,  3.9297])
tensor([-0.5320,  0.9814,  3.9297])


#### 改变形状    
用view( )来改变Tensor的形状：    
注意view( )返回的新Tensor与源Tensor虽然可能有不同的size，但是是共享data的，也即更改其中的一个，另外一个也会跟着改变。(顾名思义，view仅仅是改变了对这个张量的观察角度，内部数据并未改变)

In [13]:
y = x.view(15)
z = x.view(-1, 5)  # -1所指的维度可以根据其他维度的值推出来
print(x.size(), y.size(), z.size())

torch.Size([5, 3]) torch.Size([15]) torch.Size([3, 5])


如果我们想返回一个真正新的副本（即不共享data内存）该怎么办呢？Pytorch还提供了一个reshape()可以改变形状，但是此函数并不能保证返回的是其拷贝，所以不推荐使用。推荐先用clone创造一个副本然后再使用view。

In [14]:
# 使用clone还有一个好处是会被记录在计算图中，即梯度回传到副本时也会传到源Tensor
x_cp = x.clone().view(15)
x -= 1
print(x)
print(x_cp)

tensor([[-1.5320, -0.0186,  2.9297],
        [-0.0411, -1.1601, -1.3159],
        [-0.9924, -1.1867, -3.1629],
        [-1.5118, -2.2420, -0.4909],
        [-0.9312, -0.1049, -2.1266]])
tensor([-0.5320,  0.9814,  3.9297,  0.9589, -0.1601, -0.3159,  0.0076, -0.1867,
        -2.1629, -0.5118, -1.2420,  0.5091,  0.0688,  0.8951, -1.1266])


另外一个常用的函数就是item(), 它可以将一个标量Tensor转换成一个Python number：

In [15]:
x = torch.randn(1)
print(x)
print(x.item())

tensor([0.9127])
0.9126622080802917


##### 2.3 广播机制

In [16]:
x = torch.arange(1, 3).view(1, 2)
print(x)
y = torch.arange(1, 4).view(3, 1)
print(y)
print(x + y)

tensor([[1, 2]])
tensor([[1],
        [2],
        [3]])
tensor([[2, 3],
        [3, 4],
        [4, 5]])


##### 2.4 运算的内存开销

索引操作是不会开辟新内存的，    
而像y = x + y这样的运算是会新开内存的，然后将y指向新内存。    
为了演示这一点，我们可以使用Python自带的id函数：如果两个实例的ID一致，那么它们所对应的内存地址相同；反之则不同。

In [18]:
x = torch.tensor([1, 2])
y = torch.tensor([3, 4])
id_before = id(y)
y = y + x
print(id(y) == id_before) # False 

False


如果想指定结果到原来的y的内存，我们可以使用前面介绍的索引来进行替换操作。    
在下面的例子中，我们把x + y的结果通过[:]写进y对应的内存中。

In [20]:
x = torch.tensor([1, 2])
y = torch.tensor([3, 4])
id_before = id(y)
y[:] = y + x
print(id(y) == id_before) # True

True


还可以使用运算符全名函数中的out参数或者自加运算符+=(也即add_())达到上述效果，    
例如torch.add(x, y, out=y)和y += x(y.add_(x))。

In [21]:
x = torch.tensor([1, 2])
y = torch.tensor([3, 4])
id_before = id(y)
torch.add(x, y, out=y) # y += x, y.add_(x)
print(id(y) == id_before) # True

True


注：虽然view返回的Tensor与源Tensor是共享data的，但是依然是一个新的Tensor    
（因为Tensor除了包含data外还有一些其他属性），二者id（内存地址）并不一致。

##### 2.5 Tensor和NumPy相互转换    
我们很容易用numpy( )和from_numpy( )将Tensor和NumPy中的数组相互转换。但是需要注意的一点是： 这两个函数所产生的的Tensor和NumPy中的数组共享相同的内存（所以他们之间的转换很快），改变其中一个时另一个也会改变！    
还有一个常用的将NumPy中的array转换成Tensor的方法就是torch.tensor( ), 需要注意的是，此方法总是会进行数据拷贝（就会消耗更多的时间和空间），所以返回的Tensor和原来的数据不再共享内存。

In [22]:
# Tensor转Numpy
a = torch.ones(5)
b = a.numpy()
print(a, b)

a += 1
print(a, b)
b += 1
print(a, b)

tensor([1., 1., 1., 1., 1.]) [1. 1. 1. 1. 1.]
tensor([2., 2., 2., 2., 2.]) [2. 2. 2. 2. 2.]
tensor([3., 3., 3., 3., 3.]) [3. 3. 3. 3. 3.]


In [26]:
# Numpy转Tensor
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
print(a, b)

a += 1
print(a, b)
b += 1
print(a, b)

[1. 1. 1. 1. 1.] tensor([1., 1., 1., 1., 1.], dtype=torch.float64)
[2. 2. 2. 2. 2.] tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
[3. 3. 3. 3. 3.] tensor([3., 3., 3., 3., 3.], dtype=torch.float64)


##### 2.6 Tensor on GPU    
用方法to( )可以将Tensor在CPU和GPU（需要硬件支持）之间相互移动.

In [27]:
# 以下代码只有在PyTorch GPU版本上才会执行
if torch.cuda.is_available():
    device = torch.device("cuda")          # GPU
    y = torch.ones_like(x, device=device)  # 直接创建一个在GPU上的Tensor
    x = x.to(device)                       # 等价于 .to("cuda")
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # to()还可以同时更改数据类型

### 自动求梯度    
PyTorch提供的autograd包能够根据输入和前向传播过程自动构建计算图，并执行反向传播。本节将介绍如何使用autograd包来进行自动求梯度的有关操作。

##### 3.1 概念    
上一节介绍的Tensor是这个包的核心类，如果将其属性.requires_grad设置为True，它将开始追踪(track)在其上的所有操作（这样就可以利用链式法则进行梯度传播了）。完成计算后，可以调用.backward( )来完成所有梯度计算。此Tensor的梯度将累积到.grad属性中。    
如果不想要被继续追踪，可以调用.detach( )将其从追踪记录中分离出来，这样就可以防止将来的计算被追踪，这样梯度就传不过去了。此外，还可以用with torch.no_grad( )将不想被追踪的操作代码块包裹起来，这种方法在评估模型的时候很常用，因为在评估模型时，我们并不需要计算可训练参数（requires_grad=True）的梯度。    
Function是另外一个很重要的类。Tensor和Function互相结合就可以构建一个记录有整个计算过程的有向无环图（DAG）。每个Tensor都有一个.grad_fn属性，该属性即创建该Tensor的Function, 就是说该Tensor是不是通过某些运算得到的，若是，则grad_fn返回一个与这些运算相关的对象，否则是None。

##### 3.2 Tensor

In [28]:
# 创建一个Tensor并设置requires_grad=True
x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [29]:
y = x + 2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


注意x是直接创建的，所以它没有grad_fn, 而y是通过一个加法操作创建的，所以它有一个为<AddBackward>的grad_fn。    
像x这种直接创建的称为叶子节点，叶子节点对应的grad_fn是None。

In [30]:
print(x.is_leaf, y.is_leaf) # True False

True False


In [31]:
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


通过.requires_grad_( )来用in-place的方式改变requires_grad属性：

In [32]:
a = torch.randn(2, 2) # 缺失情况下默认 requires_grad = False
a = ((a * 3) / (a - 1))
print(a.requires_grad) # False
a.requires_grad_(True)
print(a.requires_grad) # True
b = (a * a).sum()
print(b.grad_fn)

False
True


##### 3.3 梯度

In [33]:
# 因为out是一个标量，所以调用backward()时不需要指定求导变量：
out.backward() # 等价于 out.backward(torch.tensor(1.))

In [34]:
# out关于x的梯度 d(out)/dx
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


注意：grad在反向传播过程中是累加的(accumulated)，这意味着每一次运行反向传播，梯度都会累加之前的梯度，所以一般在反向传播之前需把梯度清零。

In [35]:
# 再来反向传播一次，注意grad是累加的
out2 = x.sum()
out2.backward()
print(x.grad)

out3 = x.sum()
x.grad.data.zero_()
out3.backward()
print(x.grad)

tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])
tensor([[1., 1.],
        [1., 1.]])


现在我们解释3.1节留下的问题，为什么在y.backward()时，如果y是标量，则不需要为backward()传入任何参数；否则，需要传入一个与y同形的Tensor?     
简单来说就是为了避免向量（甚至更高维张量）对张量求导，而转换成标量对张量求导。    
举个例子，假设形状为 m x n 的矩阵 X 经过运算得到了 p x q 的矩阵 Y，Y 又经过运算得到了 s x t 的矩阵 Z。那么按照前面讲的规则，dZ/dY 应该是一个 s x t x p x q 四维张量，dY/dX 是一个 p x q x m x n的四维张量。问题来了，怎样反向传播？怎样将两个四维张量相乘？？？这要怎么乘？？？就算能解决两个四维张量怎么乘的问题，四维和三维的张量又怎么乘？导数的导数又怎么求，这一连串的问题，感觉要疯掉……     
为了避免这个问题，我们不允许张量对张量求导，只允许标量对张量求导，求导结果是和自变量同形的张量。所以必要时我们要把张量通过将所有张量的元素加权求和的方式转换为标量，举个例子，假设y由自变量x计算而来，w是和y同形的张量，则y.backward(w)的含义是：先计算l = torch.sum(y * w)，则l是个标量，然后求l对自变量x的导数。

In [36]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)


In [37]:
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float)
z.backward(v)
print(x.grad)

tensor([2.0000, 0.2000, 0.0200, 0.0020])
